In [ ]:
!pip install PyPDF2 transformers torch


In [ ]:
import PyPDF2
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from flask import Flask, request, jsonify
from pyngrok import ngrok

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Quran.pdf to Quran.pdf
Saving Bible.pdf to Bible.pdf
Saving geeta.pdf to geeta.pdf


In [ ]:


# Mount Google Drive to access the files
from google.colab import drive
drive.mount('/content/drive')

# Initialize Flask app
app = Flask(__name__)

# Initialize the model only once
model = SentenceTransformer('all-mpnet-base-v2')

# Define a function to extract text from a PDF
def pdf_to_text(pdf_path):
    """Extract text from a PDF file."""
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text

# Chunk the text for easier processing
def chunk_text(text, chunk_size=512):
    """Split the text into chunks to handle large documents."""
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= chunk_size:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk)
            current_chunk = sentence + ". "

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

# Embed the text chunks using Sentence-BERT
def create_embeddings(chunks):
    embeddings = model.encode(chunks, convert_to_tensor=False)
    return np.array(embeddings)

# Load and process the PDFs from Google Drive
print("Processing scriptures...")

# Update these paths to your PDF locations in Google Drive
geeta_text = pdf_to_text("/content/drive/MyDrive/Colab Notebooks/Books/geeta.pdf")
quran_text = pdf_to_text("/content/drive/MyDrive/Colab Notebooks/Books/Quran.pdf")
bible_text = pdf_to_text("/content/drive/MyDrive/Colab Notebooks/Books/Bible.pdf")

# Split into chunks
geeta_chunks = chunk_text(geeta_text)
quran_chunks = chunk_text(quran_text)
bible_chunks = chunk_text(bible_text)

# Create embeddings
geeta_embeddings = create_embeddings(geeta_chunks)
quran_embeddings = create_embeddings(quran_chunks)
bible_embeddings = create_embeddings(bible_chunks)

# Initialize FAISS index for efficient search
index_geeta = faiss.IndexFlatL2(geeta_embeddings.shape[1])
index_quran = faiss.IndexFlatL2(quran_embeddings.shape[1])
index_bible = faiss.IndexFlatL2(bible_embeddings.shape[1])

# Add embeddings to the FAISS index
index_geeta.add(geeta_embeddings)
index_quran.add(quran_embeddings)
index_bible.add(bible_embeddings)

print("Scriptures processed and indexed.")

# Define the search function
def search_scripture(query, index, chunks, top_k=3):
    query_embedding = model.encode([query], convert_to_tensor=False)
    distances, indices = index.search(query_embedding, top_k)
    return [chunks[i] for i in indices[0]]

# Define the Flask API
@app.route('/ask', methods=['POST'])
def ask():
    data = request.get_json()
    query = data.get('query')

    # Search scriptures for the query
    geeta_results = search_scripture(query, index_geeta, geeta_chunks)
    quran_results = search_scripture(query, index_quran, quran_chunks)
    bible_results = search_scripture(query, index_bible, bible_chunks)

    # Create a combined response
    response = {
        "Geeta": geeta_results[:1],  # Return top 1 result for simplicity
        "Quran": quran_results[:1],
        "Bible": bible_results[:1]
    }

    return jsonify(response)

# Expose the Flask app using ngrok
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

if __name__ == '__main__':
    app.run()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing scriptures...


[0, IndirectObject(18438, 0, 135532878711152)]
[0, IndirectObject(18433, 0, 135532878711152)]
[0, IndirectObject(18428, 0, 135532878711152)]
[0, IndirectObject(18423, 0, 135532878711152)]
[0, IndirectObject(18418, 0, 135532878711152)]
[0, IndirectObject(18413, 0, 135532878711152)]
[0, IndirectObject(18408, 0, 135532878711152)]
[0, IndirectObject(18403, 0, 135532878711152)]
[0, IndirectObject(18398, 0, 135532878711152)]
[0, IndirectObject(18393, 0, 135532878711152)]
[0, IndirectObject(18388, 0, 135532878711152)]
[0, IndirectObject(18383, 0, 135532878711152)]
[0, IndirectObject(18378, 0, 135532878711152)]
[0, IndirectObject(19661, 0, 135532878711152)]
[0, IndirectObject(19656, 0, 135532878711152)]
[0, IndirectObject(19651, 0, 135532878711152)]
[0, IndirectObject(19646, 0, 135532878711152)]
[0, IndirectObject(19641, 0, 135532878711152)]
[0, IndirectObject(19636, 0, 135532878711152)]
[0, IndirectObject(19631, 0, 135532878711152)]
[0, IndirectObject(19626, 0, 135532878711152)]
[0, IndirectO

In [ ]:
curl -X POST {YOUR_NGROK_URL}/ask -H "Content-Type: application/json" -d '{"query": "Tell me about life"}'


In [ ]:
!pip install flask PyPDF2 sentence-transformers faiss-cpu pyngrok


In [ ]:
import PyPDF2
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Global variables for chunks and FAISS indices
geeta_chunks, quran_chunks, bible_chunks = [], [], []
index_geeta, index_quran, index_bible = None, None, None
model = None

# One-time initialization function
def initialize():
    global geeta_chunks, quran_chunks, bible_chunks
    global index_geeta, index_quran, index_bible, model

    # Load the model
    model = SentenceTransformer('all-mpnet-base-v2')

    def pdf_to_text(pdf_path):
        """Extract text from a PDF file."""
        text = ""
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text()
        return text

    def chunk_text(text, chunk_size=512):
        """Split the text into chunks to handle large documents."""
        sentences = text.split('. ')
        chunks = []
        current_chunk = ""

        for sentence in sentences:
            if len(current_chunk) + len(sentence) <= chunk_size:
                current_chunk += sentence + ". "
            else:
                chunks.append(current_chunk)
                current_chunk = sentence + ". "

        if current_chunk:
            chunks.append(current_chunk)

        return chunks

    # Load the scriptures and preprocess them into chunks (runs once)
    geeta_text = pdf_to_text("/content/drive/MyDrive/Colab Notebooks/Books/geeta.pdf")
    quran_text = pdf_to_text("/content/drive/MyDrive/Colab Notebooks/Books/Quran.pdf")
    bible_text = pdf_to_text("/content/drive/MyDrive/Colab Notebooks/Books/Bible.pdf")

    geeta_chunks = chunk_text(geeta_text)
    quran_chunks = chunk_text(quran_text)
    bible_chunks = chunk_text(bible_text)

    # Create embeddings
    def create_embeddings(chunks):
        """Create embeddings for all chunks using SBERT."""
        embeddings = model.encode(chunks, convert_to_tensor=False)
        return np.array(embeddings)

    geeta_embeddings = create_embeddings(geeta_chunks)
    quran_embeddings = create_embeddings(quran_chunks)
    bible_embeddings = create_embeddings(bible_chunks)

    # Initialize FAISS indices for fast similarity search
    index_geeta = faiss.IndexFlatL2(geeta_embeddings.shape[1])
    index_quran = faiss.IndexFlatL2(quran_embeddings.shape[1])
    index_bible = faiss.IndexFlatL2(bible_embeddings.shape[1])

    # Add embeddings to FAISS indices
    index_geeta.add(geeta_embeddings)
    index_quran.add(quran_embeddings)
    index_bible.add(bible_embeddings)

    print("Initialization complete! Scripture texts processed and FAISS indices created.")

# Run one-time initialization
initialize()

# Function to handle queries
def search_scripture(query, top_k=3):
    """Search for the top K most relevant chunks across all scriptures."""
    global geeta_chunks, quran_chunks, bible_chunks
    global index_geeta, index_quran, index_bible, model

    # Search function
    def search_in_index(query, index, chunks):
        query_embedding = model.encode([query], convert_to_tensor=False)
        distances, indices = index.search(query_embedding, top_k)
        return [chunks[i] for i in indices[0]]

    # Query each scripture
    geeta_results = search_in_index(query, index_geeta, geeta_chunks)
    quran_results = search_in_index(query, index_quran, quran_chunks)
    bible_results = search_in_index(query, index_bible, bible_chunks)

    return {
        "Geeta": geeta_results[:top_k],
        "Quran": quran_results[:top_k],
        "Bible": bible_results[:top_k]
    }

# Example query
query = "Tell me about Life?"

# Search the scriptures
results = search_scripture(query)

# Display results
print("Geeta results:")
for res in results["Geeta"]:
    print(res)

print("\nQuran results:")
for res in results["Quran"]:
    print(res)

print("\nBible results:")
for res in results["Bible"]:
    print(res)
